In [1]:
from ConectionSQL import SQL
from Tempo import DataHora
import os
from glob import glob
import pandas as pd

sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

pd.set_option('display.max_columns',None)

data=DataHora()

In [2]:
querys={

'Romaneio':"""

IF MONTH(GETDATE())=1

WITH TabSituacaoItens (situacao,descricao) AS (

SELECT situacao,
CASE WHEN situacao='CA' THEN 'CANCELADO' WHEN situacao='DV' THEN 'DEVOLUÇÃO'
WHEN situacao='DP' THEN 'DEVOLUÇÃO PARCIAL' WHEN situacao='AB' THEN 'EM ABERTO'
ELSE 'FATURADO' END AS descricao
FROM (
SELECT DISTINCT 
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao
FROM it_pedv
INNER JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
)situacao_rom),

TabItens (nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,
ROW_NUMBER()OVER(PARTITION BY nu_ped ORDER BY nu_ped,cd_prod) AS seq,
qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,CONVERT(DECIMAL(15,4),(peso_brt*qtde)) AS peso_brt,
CONVERT(DECIMAL(15,4),(peso_liq*qtde)) AS peso_liq,TabSituacaoItens.descricao AS situacao
FROM (
SELECT it_pedv.nu_nf,it_pedv.cd_emp,
it_pedv.nu_ped,it_pedv.cd_prod,seq,
CONVERT(DECIMAL(15,4),CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END) AS qtde,
unid_vda,qtde_unid_vda,vl_unit_vda,
CONVERT(DECIMAL(15,4),(qtde_unid_vda*vl_unit_vda)) AS vl_total,produto.peso_brt,produto.peso_liq,
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao,
nota.nu_nf_emp_fat
FROM it_pedv
INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod
LEFT JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
--WHERE it_pedv.nu_nf=2950
)itens
INNER JOIN TabSituacaoItens ON itens.situacao=TabSituacaoItens.situacao),

TabDatas (cd_emp,nu_ped,cd_clien,dt_fatu) AS (

SELECT cd_emp,nu_ped,cd_clien,
CASE WHEN FATU IS NULL THEN CAPV ELSE FATU END AS dt_fatu
FROM (
SELECT cd_emp,nu_ped,cd_clien,cd_fila,
CONVERT(DATETIME,CAST(dt_criacao AS DATE),101) AS dt_criacao
FROM evento
WHERE cd_fila IN ('FATU','CAPV'))LINHA
PIVOT(MAX(dt_criacao) FOR cd_fila IN ([FATU],[CAPV]))COLUNA),

TabTPPedido (tp_ped,descricao,tipo_ped) AS (

SELECT DISTINCT
tp_ped,
descricao,
CASE WHEN bonificacao=1 THEN LEFT(descricao,CHARINDEX(' ',LTRIM(RTRIM(descricao)))) WHEN estat_com=1 THEN 'VENDAS' ELSE 'OUTROS' END AS tipo_ped
FROM tp_ped),

TabPedidos (cd_emp,nu_ped,dt_fatu,tipo_pedido,st_comercial,
cd_clien,cd_vend,nu_nf,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT ped_vda.cd_emp,ped_vda.nu_ped,TabDatas.dt_fatu,TabTPPedido.descricao,TabTPPedido.tipo_ped,
ped_vda.cd_clien,ped_vda.cd_vend,TabItens.nu_nf,
nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,TabItens.situacao
FROM ped_vda
INNER JOIN TabDatas ON ped_vda.nu_ped=TabDatas.nu_ped AND ped_vda.cd_emp=TabDatas.cd_emp AND ped_vda.cd_clien=TabDatas.cd_clien
INNER JOIN TabTPPedido ON ped_vda.tp_ped=TabTPPedido.tp_ped
INNER JOIN TabItens ON ped_vda.nu_ped=TabItens.nu_ped AND ped_vda.cd_emp=TabItens.cd_emp),

TabMotorista (cd_motor,nome,nome_resumido) AS (

SELECT cd_motor,nome,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(nome)))=0 THEN nome
ELSE LTRIM(RTRIM(LEFT(nome,CHARINDEX(' ',LTRIM(RTRIM(nome)))-1))) + ' ' + 
LTRIM(RTRIM(RIGHT(nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(nome))))-1)))
END AS nome_resumido
FROM motor
),

TabSitEntrega (situacao,st_romaneio) AS (

SELECT DISTINCT situacao,
CASE WHEN situacao='AB' THEN 'EM ABERTO' WHEN situacao='EN' THEN 'ENTREGUE' WHEN situacao='CA' THEN 'CANCELADO' WHEN situacao='PE' THEN 'PENDENTE' END 
FROM romaneio)

SELECT romaneio.nu_rom AS 'Romaneio',
CONVERT(DATETIME,CAST(dt_montagem AS DATE),101) AS 'Data da Montagem',
CONVERT(DATETIME,CAST(dt_saida AS DATE),101) AS 'Data de Saída',
CONVERT(DATETIME,CAST(dt_retorno AS DATE),101) AS 'Data do Retorno',
rot_prdf.descricao AS 'Região',romaneio.cd_motor AS 'ID Motorista',TabMotorista.nome_resumido AS 'Motorista',
veic_ent.descricao AS 'Veículo',veic_ent.placa AS 'Placa',
TabPedidos.nu_ped AS 'Pedido',nu_nf_emp_fat AS 'NFe',dt_fatu AS 'Data de Faturamento',tipo_pedido AS 'Tipo de Pedido',
st_comercial AS 'Tipo',
TabPedidos.cd_clien AS 'ID Cliente',
TabPedidos.cd_vend AS 'ID Vendedor',cd_prod AS 'SKU',
TabPedidos.seq AS 'Seq',qtde AS 'Qtde',unid_vda AS 'Unid. VDA',qtde_unid_vda AS 'Qtde VDA',vl_unit_vda AS 'Valor Unitário',
vl_total AS 'Total Vendido',TabPedidos.peso_brt AS 'Peso Bruto',TabPedidos.peso_liq AS 'Peso Líquido',
TabPedidos.situacao AS 'Situação',TabSitEntrega.st_romaneio AS 'Situação Logística'
FROM romaneio
INNER JOIN it_rom ON romaneio.nu_rom=it_rom.nu_rom
INNER JOIN TabPedidos ON it_rom.nu_nf=TabPedidos.nu_nf AND it_rom.nu_ped=TabPedidos.nu_ped
INNER JOIN rot_prdf ON romaneio.cd_rot_prdf=rot_prdf.cd_rot_prdf
INNER JOIN TabMotorista ON romaneio.cd_motor=TabMotorista.cd_motor
INNER JOIN TabSitEntrega ON romaneio.situacao=TabSitEntrega.situacao
LEFT JOIN veic_ent ON romaneio.cd_veic_ent=veic_ent.cd_veic_ent
WHERE YEAR(dt_montagem)=YEAR(GETDATE())-1 AND MONTH(dt_montagem)=12
ORDER BY romaneio.nu_rom

ELSE

WITH TabSituacaoItens (situacao,descricao) AS (

SELECT situacao,
CASE WHEN situacao='CA' THEN 'CANCELADO' WHEN situacao='DV' THEN 'DEVOLUÇÃO'
WHEN situacao='DP' THEN 'DEVOLUÇÃO PARCIAL' WHEN situacao='AB' THEN 'EM ABERTO'
ELSE 'FATURADO' END AS descricao
FROM (
SELECT DISTINCT 
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao
FROM it_pedv
INNER JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
)situacao_rom),

TabItens (nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,
ROW_NUMBER()OVER(PARTITION BY nu_ped ORDER BY nu_ped,cd_prod) AS seq,
qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,CONVERT(DECIMAL(15,4),(peso_brt*qtde)) AS peso_brt,
CONVERT(DECIMAL(15,4),(peso_liq*qtde)) AS peso_liq,TabSituacaoItens.descricao AS situacao
FROM (
SELECT it_pedv.nu_nf,it_pedv.cd_emp,
it_pedv.nu_ped,it_pedv.cd_prod,seq,
CONVERT(DECIMAL(15,4),CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END) AS qtde,
unid_vda,qtde_unid_vda,vl_unit_vda,
CONVERT(DECIMAL(15,4),(qtde_unid_vda*vl_unit_vda)) AS vl_total,produto.peso_brt,produto.peso_liq,
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao,
nota.nu_nf_emp_fat
FROM it_pedv
INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod
LEFT JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
--WHERE it_pedv.nu_nf=2950
)itens
INNER JOIN TabSituacaoItens ON itens.situacao=TabSituacaoItens.situacao),

TabDatas (cd_emp,nu_ped,cd_clien,dt_fatu) AS (

SELECT cd_emp,nu_ped,cd_clien,
CASE WHEN FATU IS NULL THEN CAPV ELSE FATU END AS dt_fatu
FROM (
SELECT cd_emp,nu_ped,cd_clien,cd_fila,
CONVERT(DATETIME,CAST(dt_criacao AS DATE),101) AS dt_criacao
FROM evento
WHERE cd_fila IN ('FATU','CAPV'))LINHA
PIVOT(MAX(dt_criacao) FOR cd_fila IN ([FATU],[CAPV]))COLUNA),

TabTPPedido (tp_ped,descricao,tipo_ped) AS (

SELECT DISTINCT
tp_ped,
descricao,
CASE WHEN bonificacao=1 THEN LEFT(descricao,CHARINDEX(' ',LTRIM(RTRIM(descricao)))) WHEN estat_com=1 THEN 'VENDAS' ELSE 'OUTROS' END AS tipo_ped
FROM tp_ped),

TabPedidos (cd_emp,nu_ped,dt_fatu,tipo_pedido,st_comercial,
cd_clien,cd_vend,nu_nf,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT ped_vda.cd_emp,ped_vda.nu_ped,TabDatas.dt_fatu,TabTPPedido.descricao,TabTPPedido.tipo_ped,
ped_vda.cd_clien,ped_vda.cd_vend,TabItens.nu_nf,
nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,TabItens.situacao
FROM ped_vda
INNER JOIN TabDatas ON ped_vda.nu_ped=TabDatas.nu_ped AND ped_vda.cd_emp=TabDatas.cd_emp AND ped_vda.cd_clien=TabDatas.cd_clien
INNER JOIN TabTPPedido ON ped_vda.tp_ped=TabTPPedido.tp_ped
INNER JOIN TabItens ON ped_vda.nu_ped=TabItens.nu_ped AND ped_vda.cd_emp=TabItens.cd_emp),

TabMotorista (cd_motor,nome,nome_resumido) AS (

SELECT cd_motor,nome,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(nome)))=0 THEN nome
ELSE LTRIM(RTRIM(LEFT(nome,CHARINDEX(' ',LTRIM(RTRIM(nome)))-1))) + ' ' + 
LTRIM(RTRIM(RIGHT(nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(nome))))-1)))
END AS nome_resumido
FROM motor
),

TabSitEntrega (situacao,st_romaneio) AS (

SELECT DISTINCT situacao,
CASE WHEN situacao='AB' THEN 'EM ABERTO' WHEN situacao='EN' THEN 'ENTREGUE' WHEN situacao='CA' THEN 'CANCELADO' WHEN situacao='PE' THEN 'PENDENTE' END 
FROM romaneio)

SELECT romaneio.nu_rom AS 'Romaneio',
CONVERT(DATETIME,CAST(dt_montagem AS DATE),101) AS 'Data da Montagem',
CONVERT(DATETIME,CAST(dt_saida AS DATE),101) AS 'Data de Saída',
CONVERT(DATETIME,CAST(dt_retorno AS DATE),101) AS 'Data do Retorno',
rot_prdf.descricao AS 'Região',romaneio.cd_motor AS 'ID Motorista',TabMotorista.nome_resumido AS 'Motorista',
veic_ent.descricao AS 'Veículo',veic_ent.placa AS 'Placa',
TabPedidos.nu_ped AS 'Pedido',nu_nf_emp_fat AS 'NFe',dt_fatu AS 'Data de Faturamento',tipo_pedido AS 'Tipo de Pedido',
st_comercial AS 'Tipo',
TabPedidos.cd_clien AS 'ID Cliente',
TabPedidos.cd_vend AS 'ID Vendedor',cd_prod AS 'SKU',
TabPedidos.seq AS 'Seq',qtde AS 'Qtde',unid_vda AS 'Unid. VDA',qtde_unid_vda AS 'Qtde VDA',vl_unit_vda AS 'Valor Unitário',
vl_total AS 'Total Vendido',TabPedidos.peso_brt AS 'Peso Bruto',TabPedidos.peso_liq AS 'Peso Líquido',
TabPedidos.situacao AS 'Situação',TabSitEntrega.st_romaneio AS 'Situação Logística'
FROM romaneio
INNER JOIN it_rom ON romaneio.nu_rom=it_rom.nu_rom
INNER JOIN TabPedidos ON it_rom.nu_nf=TabPedidos.nu_nf AND it_rom.nu_ped=TabPedidos.nu_ped
INNER JOIN rot_prdf ON romaneio.cd_rot_prdf=rot_prdf.cd_rot_prdf
INNER JOIN TabMotorista ON romaneio.cd_motor=TabMotorista.cd_motor
INNER JOIN TabSitEntrega ON romaneio.situacao=TabSitEntrega.situacao
LEFT JOIN veic_ent ON romaneio.cd_veic_ent=veic_ent.cd_veic_ent
WHERE YEAR(dt_montagem)=YEAR(GETDATE()) AND MONTH(dt_montagem)=(MONTH(GETDATE())-1)
ORDER BY romaneio.nu_rom

""",
    
'Cliente':
    
"""

WITH TabEndCli (cd_clien,seq,tp_end,cep,endereco,bairro,municipio,
estado,numero,complemento,regiao) AS (

SELECT cd_clien,seq,
tp_end,cep,endereco,bairro,municipio,estado,numero,complemento,
rot_prdf.descricao AS regiao
FROM (
SELECT cd_clien,
ROW_NUMBER()OVER(PARTITION BY cd_clien ORDER BY cd_clien) AS seq,
tp_end,cep,endereco,bairro,municipio,estado,numero,complemento
FROM end_cli)base_cli
LEFT JOIN rot_cep ON base_cli.cep BETWEEN rot_cep.cep_ini AND rot_cep.cep_fim
LEFT JOIN rot_prdf ON rot_cep.cd_rot_prdf=rot_prdf.cd_rot_prdf),

TabContato (cd_clien,tp_tel,seq,ddd,numero) AS (

SELECT cd_clien,tp_tel,seq,ddd,numero
FROM tel_cli),

TabCliDados (cd_emp,cd_clien,tabela,cond_pagto,prz_medio_max,pagto) AS (

SELECT cli_emp.cd_emp,cli_emp.cd_clien,
tab_pre.descricao AS tabela,
promocao.descricao AS cond_pagto,cli_emp.prz_medio_max,
formpgto.descricao AS pagto
FROM cli_emp
LEFT JOIN promocao ON cli_emp.seq_prom=promocao.seq_prom
LEFT JOIN tab_pre ON cli_emp.cd_tabela=tab_pre.cd_tabela
LEFT JOIN cli_emp_formpgto ON cli_emp.cd_emp=cli_emp_formpgto.cd_emp 
AND cli_emp.cd_clien=cli_emp_formpgto.cd_clien AND cli_emp_formpgto.principal=1
LEFT JOIN formpgto ON cli_emp_formpgto.formpgto=formpgto.formpgto)

SELECT * FROM (
SELECT cliente.cd_clien AS 'ID Cliente',cgc_cpf AS 'CNPJ',
LEN(cgc_cpf) AS 'CNPJ Caracter',
nome AS 'Razão Social',nome_res AS 'Nome Fantasia',tp_pes AS 'Tipo de Cliente',
CASE WHEN cliente.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS 'Status do Cliente',
ram_ativ AS 'ID Segmento',grupocli.descricao AS 'Matriz',
st_cred.descricao AS 'Crédito',
CONVERT(DATETIME,CAST(dt_cad AS DATE),101) AS 'Data de Cadastro',
CONVERT(DATETIME,CAST(dt_prim_compra AS DATE),101) AS 'Primeira Compra',
CONVERT(DATETIME,CAST(dt_ult_compra AS DATE),101) AS 'Última Compra',
DATEDIFF(DAY,
CONVERT(DATETIME,CAST(dt_ult_compra AS DATE),101),
CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)) AS 'Dias Compra',
TabCliDados.tabela AS 'Tabela',TabCliDados.cond_pagto AS 'Condição de Pagto',
TabCliDados.prz_medio_max AS 'Prazo Pagto',TabCliDados.pagto AS 'Pagamento',
TabEndCli.cep AS 'CEP',TabEndCli.endereco AS 'Endereço',TabEndCli.bairro AS 'Bairro',
TabEndCli.municipio AS 'Município',TabEndCli.numero AS 'Numero',
TabEndCli.complemento AS 'Complemento',TabEndCli.regiao AS 'Região',
TabEndCli.estado AS 'UF',
TabContato.ddd AS 'DDD',TabContato.numero AS 'Contato'
FROM cliente
LEFT JOIN TabCliDados ON cliente.cd_emp=TabCliDados.cd_emp 
AND cliente.cd_clien=TabCliDados.cd_clien
INNER JOIN st_cred ON cliente.st_cred=st_cred.st_cred
LEFT JOIN grupocli ON cliente.cd_grupocli=grupocli.cd_grupocli
LEFT JOIN TabEndCli ON TabEndCli.cd_clien=cliente.cd_clien AND TabEndCli.seq=1
LEFT JOIN TabContato ON cliente.cd_clien=TabContato.cd_clien AND TabContato.seq=1)C
/*WHERE C.[Dias Compra]<=365
ORDER BY C.[Dias Compra] DESC*/


""",
    
'Vendedor':"""


SELECT V.CD_VEND AS 'ID Vendedor',V.NOME AS 'Vendedor',
CASE WHEN 
CHARINDEX(' ',LTRIM(RTRIM(V.NOME)))=0 THEN V.NOME ELSE
LTRIM(RTRIM(LEFT(V.NOME,CHARINDEX(' ',LTRIM(RTRIM(V.NOME))))))+ ' '+
LTRIM(RTRIM(RIGHT(V.NOME,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(V.NOME))))))) 
END AS 'Nome Resumido',V.CD_EQUIPE AS 'ID Equipe',
U.E_MAIL AS 'E-mail',categ.descricao AS 'Categoria'
FROM VENDEDOR AS V
LEFT JOIN USUARIO AS U ON V.CD_VEND=U.CD_USUARIO
LEFT JOIN categ ON V.CATEG=categ.categ
--WHERE V.ATIVO=1


""",
    
'Supervisor':"""


WITH TabVendedor(cd_vend,nome,nome_resumido,st_vend,cd_equipe,e_mail,categoria) AS (

SELECT cd_vend,vendedor.nome,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))>0 THEN
LTRIM(RTRIM(LEFT(vendedor.nome,CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))))) + ' '+
LTRIM(RTRIM(RIGHT(vendedor.nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(vendedor.nome)))))))
ELSE vendedor.nome END AS nome_resumido,
CASE WHEN vendedor.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS st_vend,
cd_equipe,usuario.e_mail,categ.descricao AS	categoria
FROM vendedor
LEFT JOIN usuario ON vendedor.cd_vend=usuario.cd_usuario
LEFT JOIN categ ON vendedor.categ=categ.categ),

TabSupervisor (cd_equipe,equipe,cd_vend_sup,supervisor,email_supervisor,cd_gerente,gerente,email_gerente) AS (

SELECT supervisor.cd_equipe,supervisor.equipe,cd_vend_sup,
supervisor,email_supervisor,
supervisor.cd_gerente,TabVendedor.nome AS gerente,
TabVendedor.e_mail AS email_gerente
FROM (
SELECT equipe.cd_equipe,equipe.descricao AS equipe,cd_vend_sup,
TabVendedor.nome_resumido AS supervisor,e_mail AS email_supervisor,
gerencia.descricao AS cd_gerente
FROM equipe
INNER JOIN TabVendedor ON equipe.cd_vend_sup=TabVendedor.cd_vend
INNER JOIN gerencia ON equipe.cd_gerencia=gerencia.cd_gerencia
)supervisor
INNER JOIN TabVendedor ON supervisor.cd_gerente=TabVendedor.cd_vend)

SELECT cd_equipe AS 'ID Equipe',equipe AS 'Equipe',cd_vend_sup AS 'ID Sup',
supervisor AS 'Supervisor',email_supervisor AS 'Email Sup',
cd_gerente AS 'ID Gerente',gerente AS 'Gerente',email_gerente AS 'Email Gerente'
FROM TabSupervisor


""",
    
'Frete':"""


WITH TabSituacaoItens (situacao,descricao) AS (

SELECT situacao,
CASE WHEN situacao='CA' THEN 'CANCELADO' WHEN situacao='DV' THEN 'DEVOLUÇÃO'
WHEN situacao='DP' THEN 'DEVOLUÇÃO PARCIAL' WHEN situacao='AB' THEN 'EM ABERTO'
ELSE 'FATURADO' END AS descricao
FROM (
SELECT DISTINCT 
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao
FROM it_pedv
INNER JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
)situacao_rom),

TabItens (nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT nu_nf,cd_emp,nu_ped,nu_nf_emp_fat,cd_prod,
ROW_NUMBER()OVER(PARTITION BY nu_ped ORDER BY nu_ped,cd_prod) AS seq,
qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,CONVERT(DECIMAL(15,4),(peso_brt*qtde)) AS peso_brt,
CONVERT(DECIMAL(15,4),(peso_liq*qtde)) AS peso_liq,TabSituacaoItens.descricao AS situacao
FROM (
SELECT it_pedv.nu_nf,it_pedv.cd_emp,
it_pedv.nu_ped,it_pedv.cd_prod,seq,
CONVERT(DECIMAL(15,4),CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END) AS qtde,
unid_vda,qtde_unid_vda,vl_unit_vda,
CONVERT(DECIMAL(15,4),(qtde_unid_vda*vl_unit_vda)) AS vl_total,produto.peso_brt,produto.peso_liq,
CASE WHEN it_pedv.situacao=nota.situacao THEN it_pedv.situacao
WHEN it_pedv.situacao='DV' AND nota.situacao='DP' THEN 'DP' ELSE it_pedv.situacao END AS situacao,
nota.nu_nf_emp_fat
FROM it_pedv
INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod
LEFT JOIN nota ON it_pedv.nu_nf=nota.nu_nf AND it_pedv.cd_emp=nota.cd_emp AND it_pedv.nu_ped=nota.nu_ped
--WHERE it_pedv.nu_nf=2950
)itens
INNER JOIN TabSituacaoItens ON itens.situacao=TabSituacaoItens.situacao),

TabDatas (cd_emp,nu_ped,cd_clien,dt_fatu) AS (

SELECT cd_emp,nu_ped,cd_clien,
CASE WHEN FATU IS NULL THEN CAPV ELSE FATU END AS dt_fatu
FROM (
SELECT cd_emp,nu_ped,cd_clien,cd_fila,
CONVERT(DATETIME,CAST(dt_criacao AS DATE),101) AS dt_criacao
FROM evento
WHERE cd_fila IN ('FATU','CAPV'))LINHA
PIVOT(MAX(dt_criacao) FOR cd_fila IN ([FATU],[CAPV]))COLUNA),

TabTPPedido (tp_ped,descricao,tipo_ped) AS (

SELECT DISTINCT
tp_ped,
descricao,
CASE WHEN bonificacao=1 THEN LEFT(descricao,CHARINDEX(' ',LTRIM(RTRIM(descricao)))) WHEN estat_com=1 THEN 'VENDAS' ELSE 'OUTROS' END AS tipo_ped
FROM tp_ped),

TabPedidos (cd_emp,nu_ped,dt_fatu,tipo_pedido,st_comercial,
cd_clien,cd_vend,nu_nf,nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,situacao) AS (

SELECT ped_vda.cd_emp,ped_vda.nu_ped,TabDatas.dt_fatu,TabTPPedido.descricao,TabTPPedido.tipo_ped,
ped_vda.cd_clien,ped_vda.cd_vend,TabItens.nu_nf,
nu_nf_emp_fat,cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,
vl_total,peso_brt,peso_liq,TabItens.situacao
FROM ped_vda
INNER JOIN TabDatas ON ped_vda.nu_ped=TabDatas.nu_ped AND ped_vda.cd_emp=TabDatas.cd_emp AND ped_vda.cd_clien=TabDatas.cd_clien
INNER JOIN TabTPPedido ON ped_vda.tp_ped=TabTPPedido.tp_ped
INNER JOIN TabItens ON ped_vda.nu_ped=TabItens.nu_ped AND ped_vda.cd_emp=TabItens.cd_emp),

TabFrete (nu_rom,peso_brt,peso_liq,vl_frete_pago,cuto_brt) AS (

SELECT nu_rom,peso_brt,peso_liq,vl_frete_pago,
vl_frete_pago/NULLIF(peso_brt,0) AS cuto_brt
FROM (
SELECT romaneio.nu_rom,SUM(peso_brt) AS peso_brt,SUM(peso_liq) AS peso_liq,vl_frete_pago
FROM romaneio
INNER JOIN it_rom ON romaneio.nu_rom=it_rom.nu_rom
INNER JOIN TabPedidos ON it_rom.nu_nf=TabPedidos.nu_nf AND it_rom.nu_ped=TabPedidos.nu_ped
GROUP BY romaneio.nu_rom,vl_frete_pago)roteiro)

SELECT nu_rom AS 'Romaneio',cuto_brt AS 'Custo KG'
FROM TabFrete
ORDER BY nu_rom


""",
    
'Produto':"""


WITH TabCaixa (cd_prod,qtde_caixa) AS (

SELECT produto.cd_prod,
CASE WHEN unid_prod.qtde_unid IS NULL THEN 0 ELSE unid_prod.qtde_unid END AS qtde_caixa
FROM produto
LEFT JOIN unid_prod ON produto.cd_prod=unid_prod.cd_prod AND unid_prod.unid_vda='CX')

SELECT produto.cd_prod AS 'SKU',produto.cd_prod_fabric AS 'Cód. Fabricante',produto.descricao AS 'Produto',
CASE WHEN produto.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END 'Status',fabric.descricao AS 'Fabricante',
depto.descricao AS 'Departamento',secao.descricao AS 'Seção',categprd.descricao AS 'Categoria',linha.descricao AS 'Linha'
FROM produto
INNER JOIN fabric ON produto.cd_fabric=fabric.cd_fabric
INNER JOIN linha ON produto.cd_linha=dbo.linha.cd_linha 
INNER JOIN categprd ON linha.cd_categprd=dbo.categprd.cd_categprd
INNER JOIN secao ON linha.cd_secao=dbo.secao.cd_secao 
INNER JOIN depto ON secao.cd_depto=dbo.depto.cd_depto
INNER JOIN TabCaixa ON produto.cd_prod=TabCaixa.cd_prod


"""

    
}

In [3]:
tabelas_df=dict()


for tabela,query in querys.items():
    
    tabelas_df[tabela]=pd.read_sql(query,conectando)
    
    pass

In [4]:
tabelas_df['Romaneio'].columns

Index(['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente',
       'ID Vendedor', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação', 'Situação Logística'],
      dtype='object')

In [5]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria'],
      dtype='object')

In [6]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [7]:
tabelas_df['Frete'].columns

Index(['Romaneio', 'Custo KG'], dtype='object')

In [8]:
tabelas_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha'],
      dtype='object')

In [9]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato'],
      dtype='object')

In [10]:
tabelas_df['Romaneio']=tabelas_df['Romaneio'].merge(tabelas_df['Frete'],on='Romaneio',how='inner')[['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente',
       'ID Vendedor', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação','Situação Logística','Custo KG']]

In [11]:
tabelas_df['Romaneio']['Frete R$']=tabelas_df['Romaneio'].apply(lambda info: info['Custo KG']*info['Peso Bruto'],axis=1)

In [12]:
tabelas_df['Romaneio'][['Romaneio','Peso Bruto','Frete R$']].groupby(['Romaneio']).sum()

,Peso Bruto,Frete R$
Romaneio,,
31052,3.780,0.0
31053,851.520,350.0
31054,566.028,330.0
31055,689.620,370.0
31056,534.030,500.0
...,...,...
31656,756.508,420.0
31658,148.200,250.0
31659,433.805,330.0


In [13]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato'],
      dtype='object')

In [14]:
tabelas_df['Romaneio'].rename(columns={'Região':'Região do Roteiro'},inplace=True)

In [15]:
tabelas_df['Romaneio']=tabelas_df['Romaneio'].merge(tabelas_df['Cliente'],on='ID Cliente',how='inner')[['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região do Roteiro', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente','CNPJ', 'Razão Social', 'Nome Fantasia',
       'ID Vendedor', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação','Situação Logística','Custo KG','Frete R$']]

In [16]:
tabelas_df['Romaneio']=tabelas_df['Romaneio'].merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região do Roteiro', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente','CNPJ', 'Razão Social', 'Nome Fantasia',
       'ID Vendedor','Vendedor', 'Nome Resumido', 'ID Equipe', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação','Situação Logística','Custo KG','Frete R$']]

In [17]:
tabelas_df['Romaneio']=tabelas_df['Romaneio'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região do Roteiro', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente','CNPJ', 'Razão Social', 'Nome Fantasia',
       'ID Vendedor','Vendedor', 'Nome Resumido', 'Equipe','Supervisor', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação','Situação Logística','Custo KG','Frete R$']]

In [18]:
tabelas_df['Romaneio']=tabelas_df['Romaneio'].merge(tabelas_df['Produto'],on='SKU',how='inner')[['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região do Roteiro', 'ID Motorista', 'Motorista', 'Veículo', 'Placa', 'Pedido',
       'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo', 'ID Cliente','CNPJ', 'Razão Social', 'Nome Fantasia',
       'ID Vendedor','Vendedor', 'Nome Resumido', 'Equipe','Supervisor', 'SKU','Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor Unitário', 'Total Vendido', 'Peso Bruto', 'Peso Líquido',
       'Situação','Situação Logística','Custo KG','Frete R$']]


In [19]:
data_atual=data.HoraAtual()

In [20]:
base_df=tabelas_df['Romaneio'].loc[(tabelas_df['Romaneio']['Data da Montagem'].dt.year==data_atual.year)&(tabelas_df['Romaneio']['Data da Montagem'].dt.month==(data_atual.month-1))]

In [24]:
base_df.columns

Index(['Romaneio', 'Data da Montagem', 'Data de Saída', 'Data do Retorno',
       'Região do Roteiro', 'ID Motorista', 'Motorista', 'Veículo', 'Placa',
       'Pedido', 'NFe', 'Data de Faturamento', 'Tipo de Pedido', 'Tipo',
       'ID Cliente', 'CNPJ', 'Razão Social', 'Nome Fantasia', 'ID Vendedor',
       'Vendedor', 'Nome Resumido', 'Equipe', 'Supervisor', 'SKU', 'Produto',
       'Status', 'Fabricante', 'Departamento', 'Seção', 'Categoria', 'Linha',
       'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor Unitário',
       'Total Vendido', 'Peso Bruto', 'Peso Líquido', 'Situação',
       'Situação Logística', 'Custo KG', 'Frete R$'],
      dtype='object')

In [21]:
motorista=base_df[['ID Motorista','Motorista','Total Vendido','Frete R$']].groupby(['ID Motorista','Motorista'],as_index=False).sum()

frete=motorista['Frete R$'].sum()

valor=motorista['Total Vendido'].sum()

nfe=len(base_df['NFe'].unique().tolist())

pedido=len(base_df['Pedido'].unique().tolist())

produto=len(base_df['SKU'].unique().tolist())

caminhao=len(base_df['Veículo'].unique().tolist())

cliente=len(base_df['ID Cliente'].unique().tolist())

In [30]:
motorista['Qtde NFe']=motorista.apply(lambda info: len(base_df['NFe'].loc[base_df['ID Motorista']==info['ID Motorista']].unique().tolist()),axis=1)

motorista['Qtde Cliente']=motorista.apply(lambda info: len(base_df['ID Cliente'].loc[base_df['ID Motorista']==info['ID Motorista']].unique().tolist()),axis=1)

motorista['Qtde Produto']=motorista.apply(lambda info: len(base_df['SKU'].loc[base_df['ID Motorista']==info['ID Motorista']].unique().tolist()),axis=1)

motorista

,ID Motorista,Motorista,Total Vendido,Frete R$,Qtde NFe,Qtde Cliente,Qtde Produto
0,00000000,PROVISORIO,87744.4783,0.0,42,18,83
1,ALCIDES,ALCIDES LIM,99262.7100,4490.0,195,148,160
2,ALVES,EDSON OLIVEIRA,88246.8618,4270.0,177,141,166
3,CALUDEMI,CLAUDEMIR BERTASSO,311385.8700,13200.0,111,61,107
4,CARLOS,CARLOS SILVA,223481.7500,8610.0,437,196,203
5,CLAYTON,CLAYTON SOUSA,106332.7100,5536.0,130,107,141
6,DEIVID,DEIVID RODRIGUES,339294.5658,12870.0,681,316,228
7,EDGAR,EDGARD CARVALHO,197278.8361,8920.0,353,253,190
8,EDSON,EDSON NET,345.6800,0.0,1,1,2
9,FARIAS,LUIZ FARIAS,78730.6916,3380.0,81,71,85
